<a href="https://colab.research.google.com/github/Starksood/Experimental_Conundrums/blob/main/MultiClass_Classification_Rock_Paper_Scissocs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
#Using the dataset is simple. Download and unzip it—the sorted 
#subdirectories are  already present in the ZIP file—and then use it
# to initialize an ImageDataGenerator:

!wget --no-check-certificate \
https://storage.googleapis.com/laurencemoroney-blog.appspot.com/rps.zip \
-O /tmp/rps.zip
local_zip = '/tmp/rps.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp/')
zip_ref.close()
TRAINING_DIR = "/tmp/rps/"
training_datagen = ImageDataGenerator(
rescale = 1./255,
rotation_range=40,
width_shift_range=0.2,
height_shift_range=0.2,
shear_range=0.2,
zoom_range=0.2,
horizontal_flip=True,
fill_mode='nearest'
)
#Note, however, that when you set up the data generator 
#from this, you have to specify that the class mode is categorical
# in order for the ImageDataGenerator to use more than two subdirectories:

train_generator = training_datagen.flow_from_directory(
TRAINING_DIR,
target_size=(150,150),
class_mode='categorical'
)

#When defining your model, while keeping an eye on the input and output layers, you
#want to ensure that the input matches the shape of the data (in this case 150 × 150)
#and that the output matches the number of classes (now three):

model = tf.keras.models.Sequential([
# Note the input shape is the desired size of the image:
# 150x150 with 3 bytes color
# This is the first convolution
tf.keras.layers.Conv2D(64, (3,3), activation='relu',
input_shape=(150, 150, 3)),
tf.keras.layers.MaxPooling2D(2, 2),
# The second convolution
tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
tf.keras.layers.MaxPooling2D(2,2),
# The third convolution
tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
tf.keras.layers.MaxPooling2D(2,2),
# The fourth convolution
tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
tf.keras.layers.MaxPooling2D(2,2),
# Flatten the results to feed into a DNN
tf.keras.layers.Flatten(),
# 512 neuron hidden layer
tf.keras.layers.Dense(512, activation='relu'),
tf.keras.layers.Dense(3, activation='softmax')
])

# Finally, when compiling your model, you want to ensure that it uses a categorical loss
#function, such as categorical cross entropy. Binary cross entropy will not work with more than two classes:
model.compile(loss = 'categorical_crossentropy', optimizer='rmsprop',
metrics=['accuracy'])
# Training is then the same as before:
history = model.fit(train_generator, epochs=25,
validation_data = validation_generator, verbose = 1)

# Your code for testing predictions will also need to change somewhat. There are now
#three output neurons, and they will output a value close to 1 for the predicted class,
#which will ensure that all three predictions will add up to 1. For example, if the model
#sees something it’s really unsure about it might output .4, .4, .2, but if it sees something
#it’s quite sure about you might get .98, .01, .01.
#Note also that when using the ImageDataGenerator, the classes are loaded in alphabetical
#order—so while you might expect the output neurons to be in the order of the
#name of the game, the order in fact will be Paper, Rock, Scissors.

# Code to try out predictions in a Colab notebook will look like this. It’s very similar to
# what you saw earlier:
 import numpy as np
from google.colab import files
 from keras.preprocessing import image
uploaded = files.upload()
     for fn in uploaded.keys():
# predicting images
path = fn
img = image.load_img(path, target_size=(150, 150))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
images = np.vstack([x])
classes = model.predict(images, batch_size=10)
print(fn)
print(classes)

#You can see from the filenames what the images were. Paper1.png ended up as [1, 0,
#0], meaning the first neuron was activated and the others weren’t. Similarly,
#Rock1.png ended up as [0, 1, 0], activating the second neuron, and Scissors2.png
#was [0, 0, 1]. Remember that the neurons are in alphabetical order by label!
#Some images that you can use to test the dataset are available to download. Alternatively,
#of course, you can try your own. Note that the training images are all done against a plain white background, though, so there may be some confusion if there is
#a lot of detail in the background of the photos you take.




IndentationError: ignored